In [1]:
from bbi.models import ExpectationModel
import numpy as np

In [2]:
env = ExpectationModel(has_state_offset=False)
env.reset()

(array([0., 5., 0., 0.], dtype=float32), {})

In [3]:
env.set_state(
    state=np.array([10.0, 5.0, 0.0, 0.0]),
    previous_status=0
)
env.state

array([10.,  5.,  0.,  0.])

In [18]:
env.step(1)

(array([10.        ,  5.        ,  0.33333333,  0.33333333]),
 -1.0,
 False,
 False,
 {})

In [28]:
env._get_observation()

array([10.        ,  5.        ,  0.33333334,  0.33333334], dtype=float32)